In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

## Check clip sizes

In [2]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'somali', 'build') 

In [10]:
import os, pickle
from multiprocessing import Pool
from RecordingCorpus import RecordingCorpus

In [12]:
import sys, pickle, os, warnings
warnings.filterwarnings("ignore")

In [13]:
language='somali'
with Pool(16) as pool:
    recordings = RecordingCorpus(C, pool)

100%|██████████| 132/132 [00:11<00:00, 11.43it/s]


In [14]:
from SplitCorpus import SplitCorpus
splits=SplitCorpus.transcript_split(C, recordings)

In [16]:
vars(splits.artifacts[0])

{'key': ('somali',
  'BABEL_OP3_405_68059_20141209_220426_inLine',
  (29360, 43120)),
 'source': <AudioArtifact.AudioArtifact at 0x7fcfae6c6410>,
 'target': <PhraseArtifact.PhraseArtifact at 0x7fcfae6c63d0>}

In [7]:
import numpy as np

In [3]:
splits=C.split_files()

In [9]:
len(splits)

19734

In [17]:
L=[x.source.n_seconds for x in splits.artifacts]
min(L), np.mean(L), np.median(L), max(L),sum(L)/len(L)

(0.135, 3.552724966646902, 2.52, 45.58, 3.5527249666468856)

In [22]:
len([x for x in L if x < 10])/len(L)

0.9363573475639886

In [24]:
for x in splits.artifacts:
    break

In [32]:
V=list(sorted(set(''.join([x.target.value.lower() for x in splits.artifacts]))))

In [33]:
V

[' ',
 "'",
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

## Train step 0: Start from scratch (constrained case)

In [34]:
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2020-11-03 23:18:14 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 23:18:14 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 23:18:14 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 23:18:14 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-03 23:18:14 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

In [35]:
import logging
logging.getLogger('nemo_logger').setLevel(logging.ERROR)

In [37]:
from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = f'{C.language}_{C.sample_rate}.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)

MissingMandatoryValue: Missing mandatory value: train_ds.manifest_filepath
	full_key: train_ds.manifest_filepath
	reference_type=Any
	object_type=dict

In [19]:
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest
model.change_vocabulary(new_vocabulary=params['labels'])

[NeMo I 2020-10-19 11:44:46 ctc_models:189] Changed decoder to output to [' ', "'", '-', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Á', 'Â', 'Ú', 'à', 'á', 'â', 'ã', 'è', 'é', 'ê', 'ì', 'í', 'ò', 'ó', 'ô', 'õ', 'ù', 'ú', 'ý', 'ă', 'Đ', 'đ', 'ĩ', 'ũ', 'ơ', 'ư', 'ạ', 'Ả', 'ả', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ẹ', 'ẻ', 'ẽ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ỉ', 'ị', 'ọ', 'ỏ', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'ụ', 'ủ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ỳ', 'ỷ', 'ỹ'] vocabulary.


## Train step 0: Load most recent trained model

In [ ]:
from load_pretrained_model import load_pretrained_model
model = load_pretrained_model(C, 0)

from reshuffle_samples import reshuffle_samples
reshuffle_samples(C)

from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = f'{C.language}_{C.sample_rate}.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest

## Train step 1: Train

In [21]:
import pytorch_lightning as pl
import os, datetime

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=C.model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=1000, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [23]:
model.set_trainer(trainer)
model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

[NeMo I 2020-10-19 11:55:27 collections:173] Dataset loaded with 9158 files totalling 9.87 hours
[NeMo I 2020-10-19 11:55:27 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-19 11:55:27 collections:173] Dataset loaded with 1018 files totalling 1.10 hours
[NeMo I 2020-10-19 11:55:27 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-19 11:55:27 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 5e-05
        weight_decay: 0.001
    )
[NeMo I 2020-10-19 11:55:27 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fb0abd65310>" 
    will be used during training (effective maximum steps = 381583) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 381583
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 5e-05
     lr: 5e-05
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7fb0abd65310>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'val_loss',
  'reduce_on_plateau': False})

In [24]:
trainer.fit(model)

[NeMo I 2020-10-19 11:55:39 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 5e-05
        weight_decay: 0.001
    )
[NeMo I 2020-10-19 11:55:39 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fb0abf22a10>" 
    will be used during training (effective maximum steps = 381583) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 381583
    )


[NeMo W 2020-10-19 11:55:39 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:
    
    Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
    
    

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | _wer              | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 130 K 
5 | loss              | CTCLoss                           | 0     


[NeMo I 2020-10-19 11:55:42 wer:148] 
    
[NeMo I 2020-10-19 11:55:42 wer:149] reference:mấy cái cửa hàng cửa hàng bữa nay có cái hãng hát K phôn ấy
[NeMo I 2020-10-19 11:55:42 wer:150] decoded  :rqrVớờÚểớdổéảéẻừặửéớLdôõKảéẻéÚựáựVrxềủdÚrKÚgdẻrừdÚrVéVéráqVqrqáVrữứửéờặờéIréạdọéđtẻtdédéởétéễtéểôvégMéõéõéờtégrédỉDựớáỡ


[NeMo W 2020-10-19 11:55:43 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning:
    
    Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
    
    


[NeMo I 2020-10-19 11:56:05 wer:148] 
    
[NeMo I 2020-10-19 11:56:05 wer:149] reference:thế à
[NeMo I 2020-10-19 11:56:05 wer:150] decoded  :rVáVôủdVÚVdtôtsôừdảdôéảọảọdG
[NeMo I 2020-10-19 11:56:27 wer:148] 
    
[NeMo I 2020-10-19 11:56:27 wer:149] reference:ờ rồi sao nữa rồi sao hả
[NeMo I 2020-10-19 11:56:27 wer:150] decoded  :rừĐrĐừĐgĐrÚéÚgừxdôVrgrừrgâềôgdgrgểgdéógềéLủtgữrgéõéặrG
[NeMo I 2020-10-19 11:56:50 wer:148] 
    
[NeMo I 2020-10-19 11:56:50 wer:149] reference:tính dừa tươi tao đi lộc về cho mày uống rồi tao cất dần tới thứ thứ Bảy Chủ nhật tuần sau đó là dữ lắm chứ không thôi là sợ hư á
[NeMo I 2020-10-19 11:56:50 wer:150] decoded  :ởrgtétgG
[NeMo I 2020-10-19 11:57:12 wer:148] 
    
[NeMo I 2020-10-19 11:57:12 wer:149] reference:đi đâu
[NeMo I 2020-10-19 11:57:12 wer:150] decoded  :ởrVừrừrVừ ôVôảôdủdédrG
[NeMo I 2020-10-19 11:57:35 wer:148] 
    
[NeMo I 2020-10-19 11:57:35 wer:149] reference:nhưng thế trường em dãy bàn nó chia kiểu gì bốn dãy bàn à hay hai dãy
[NeMo I 

...

[NeMo I 2020-10-19 12:16:39 wer:148] 
    
[NeMo I 2020-10-19 12:16:39 wer:149] reference:thời vụ à
[NeMo I 2020-10-19 12:16:39 wer:150] decoded  : é



Epoch 00006: saving model to /home/catskills/Desktop/openasr20/save/nemo_vietnamese/amharic_20201019_114603_394609_epoch=06.ckpt


[NeMo I 2020-10-19 12:17:03 wer:148] 
    
[NeMo I 2020-10-19 12:17:03 wer:149] reference:vâng vâng vâng vâng
[NeMo I 2020-10-19 12:17:03 wer:150] decoded  : é
[NeMo I 2020-10-19 12:17:26 wer:148] 
    
[NeMo I 2020-10-19 12:17:26 wer:149] reference:ừ nghe được
[NeMo I 2020-10-19 12:17:26 wer:150] decoded  : é
[NeMo I 2020-10-19 12:17:50 wer:148] 
    
[NeMo I 2020-10-19 12:17:50 wer:149] reference:ờ
[NeMo I 2020-10-19 12:17:50 wer:150] decoded  :ừé


[NeMo W 2020-10-19 12:18:09 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:
    
    Detected KeyboardInterrupt, attempting graceful shutdown...
    
    
Saving latest checkpoint..


1

## DEV translation

In [ ]:
from Cfg import Cfg
from glob import glob
from package_DEV import package_DEV
from load_pretrained_amharic_model import load_pretrained_amharic_model
version='113'
C = Cfg('NIST', 16000, 'pashto', 'dev', version)
model = load_pretrained_amharic_model(C, 0)
files=list(sorted(glob(f'{C.audio_split_dir}/*.wav')))
translations=model.transcribe(paths2audio_files=files, batch_size=32)